# ======================================
# 🏥 인천 노인 건강 도우미 (Jupyter Notebook 버전)
# ======================================

In [46]:
# 필수 패키지 설치
!pip install transformers torch sentencepiece pandas

import re
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [47]:
# -----------------------------
# ✅ 1. 데이터 불러오기
# -----------------------------
try:
    health_df = pd.read_csv("./data/국민건강보험공단_건강검진정보_2024.CSV", encoding="cp949", nrows=100000)
    print("✅ 건강검진 데이터 로드 완료")
except Exception as e:
    print("⚠️ 건강검진 데이터 로드 실패:", e)
    health_df = None

try:
    hosp_df = pd.read_csv("./data/incheon_health_check_centers.csv", encoding="utf-8-sig")
    print("✅ 인천 검진기관 데이터 로드 완료")
except Exception as e:
    print("⚠️ 병원 데이터 로드 실패:", e)
    hosp_df = pd.DataFrame({
        "name": ["인천의료원", "인하대병원", "인천성모병원"],
        "phone": ["032-580-6000", "032-890-2114", "032-1544-9004"]
    })


✅ 건강검진 데이터 로드 완료
✅ 인천 검진기관 데이터 로드 완료


In [48]:
# -----------------------------
# ✅ 2. T5 모델 로드
# -----------------------------
print("🔄 T5 모델 로드 중... (KETI-AIR/ke-t5-base)")
tokenizer = T5Tokenizer.from_pretrained("KETI-AIR/ke-t5-base")
model = T5ForConditionalGeneration.from_pretrained("KETI-AIR/ke-t5-base")
print("✅ 모델 로드 완료")

🔄 T5 모델 로드 중... (KETI-AIR/ke-t5-base)
✅ 모델 로드 완료


In [51]:
# -----------------------------
# ✅ 3. 텍스트 정제 및 보조 함수
# -----------------------------
def is_garbled(text: str) -> bool:
    """T5가 잘못 출력한 무의미한 반복 문자열 감지"""
    if not text or len(text) < 10:
        return True
    if re.search(r'(.{2,6})\1{3,}', text):  # 반복 패턴
        return True
    hangul = sum(1 for c in text if '\uac00' <= c <= '\ud7a3')
    if len(text) > 0 and (hangul / len(text)) < 0.5:
        return True
    return False


def fallback_schedule(age, gender):
    """T5가 오류 시 기본 검진 안내문 제공"""
    exams = ["기본 건강검진"]
    if age >= 40: exams.append("위암 검진")
    if age >= 50: exams.append("대장암 검진")
    if age >= 66: exams.append("치매 조기검진")
    if gender == "여성" and age >= 40: exams.append("유방암 검진")
    if gender == "남성" and age >= 50: exams.append("전립선 검진")

    return f"{age}세 {gender}은(는) {', '.join(exams)} 대상입니다. 혈압과 혈당을 관리하고, 검진 전날은 금식하세요."


def nearby_hospitals():
    """인근 검진 기관 3곳 안내"""
    text = "📍 인근 검진기관:\n"
    for _, r in hosp_df.head(3).iterrows():
        text += f"- {r.get('name','기관')} ({r.get('phone','전화번호')})\n"
    return text.strip()


def generate_answer(prompt, age=None, gender=None):
    """T5로 답변 생성 후 필터링"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_new_tokens=120, num_beams=2, no_repeat_ngram_size=3)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # 🚫 무의미한 반복/영문 잡음 제거
    if is_garbled(text):
        return fallback_schedule(age, gender) + "\n\n" + nearby_hospitals()
    else:
        # 특정 잡음 필터링
        clean = re.sub(r"(큐브|쿨|젊|베네|비일비재|Panther|Colbert|scoreless|기자|주신고가|명희숙|Papa|Hyper|광복|peer)+", "", text)
        clean = re.sub(r"\s+", " ", clean).strip()
        return clean + "\n\n" + nearby_hospitals()


In [52]:
# -----------------------------
# ✅ 4. 대화형 챗봇 시뮬레이터
# -----------------------------
def chatbot():
    print("\n🧓 인천 노인 건강 도우미 챗봇입니다.")
    print("--------------------------------------------------")
    print("원하시는 정보를 선택하세요:")
    print("1) 검진 일정")
    print("2) 건강관리 팁")
    print("3) 검진 준비방법")
    print("0) 종료")
    print("--------------------------------------------------")

    while True:
        choice = input("👉 메뉴 선택: ").strip()
        if choice == "0":
            print("안전하게 종료합니다. 건강하세요!")
            break

        age_input = input("나이 (예: 70): ").strip()
        gender_input = input("성별 (남성/여성): ").strip()

        try:
            age = int(re.search(r'(\d{2,3})', age_input).group(1))
        except:
            age = 0
        gender = gender_input if gender_input in ["남성", "여성"] else "모름"

        if choice == "1":
            prompt = f"{age}세 {gender}이(가) 받을 수 있는 건강검진 항목과 검진 주기를 2문장으로 설명해줘."
        elif choice == "2":
            prompt = f"{age}세 {gender}에게 혈압, 혈당, 지질, 체중, 영양, 수면, 스트레스 등 건강관리 팁을 알려줘."
        elif choice == "3":
            prompt = f"{age}세 {gender}이(가) 건강검진을 받기 전 준비해야 할 사항을 간단히 안내해줘."
        else:
            print("⚠️ 잘못된 입력입니다. 다시 선택해주세요.")
            continue

        answer = generate_answer(prompt, age, gender)
        print("\n🤖 답변:")
        print(answer)
        print("--------------------------------------------------\n")


In [ ]:
# -----------------------------
# ✅ 5. 실행
# -----------------------------
chatbot()


🧓 인천 노인 건강 도우미 챗봇입니다.
--------------------------------------------------
원하시는 정보를 선택하세요:
1) 검진 일정
2) 건강관리 팁
3) 검진 준비방법
0) 종료
--------------------------------------------------
⚠️ 잘못된 입력입니다. 다시 선택해주세요.
⚠️ 잘못된 입력입니다. 다시 선택해주세요.
⚠️ 잘못된 입력입니다. 다시 선택해주세요.
⚠️ 잘못된 입력입니다. 다시 선택해주세요.
⚠️ 잘못된 입력입니다. 다시 선택해주세요.
⚠️ 잘못된 입력입니다. 다시 선택해주세요.
⚠️ 잘못된 입력입니다. 다시 선택해주세요.
⚠️ 잘못된 입력입니다. 다시 선택해주세요.
